# MSCF 46982 Market Microstructure and Algorithmic Trading
# Fall 2018 Mini 2

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", specify your name and that of your collaborator, and remove the `notimplemented` exception.



---

In [1]:
NAME: "Ze Yang"
COLLABORATOR: ""

## Market Impact

Deciding when and how to trade large orders is fundamental to designing algorithmic trading engines (algos).  VWAP and TWAP were some of the first algos.  To design algos effectively, volume, spread and volatility profiles (built in HW3) are needed.

We now turn our attention to the cost that comes with every trading decision - the market impact.

This assignment will walk you through querying and efficiently merging a large trade and quote dataset, using the [Lee and Ready (1991)](https://onlinelibrary.wiley.com/doi/epdf/10.1111/j.1540-6261.1991.tb02683.x) algorithm to determine if the trades are buyer or seller initiated, and finally using regression to compute ([Kyle's lambda 1985](https://www.rhsmith.umd.edu/files/Documents/Centers/CFP/research/kyle1985.pdf)) as a measure if each stock's liquidity. The final question asks you to predict market impact using the Square Root Law.

The previous assignment focused on locally analyzing the raw trade and quote data.  Now that our analysis is growing, this is not be possible.  This assignment pushes much of the heavy lifting back to the Kdb+ server and returns summarized data to your Jupyter Notebook for further analysis.

We start by opening a connection to the NYSE Daily TAQ database. The generated integer saved in `h` is a Kdb+ file handle.  It will be used for all database communication.

NOTE: The database is located on a CMU server - behind the firewall.  If you are doing this assignment from home, you will need to connect to the CMU network using the Cisco AnyConnect VPN software.

In [2]:
\c 5 100
h:0N!hopen `$":tpr-mscf-kx.tepper.cmu.edu:5000:mscf2018:mmat46982"

8i


### Part A (1 points)

Complete the `tradequery` function so that it:

1. **Selects trade information for a list of stocks for a single day and the given time range and trade conditions.**

Given a start time, end time, list of symbol and string of desired trade conditions, return a table with only the date,time,sym,price and size columns from the `trade` table.

In [3]:
/ (s)tart (t)ime, (e)nd (t)ime, list of (s)ym, trade (c)ondition codes, (d)a(t)e
tradequery:{[st;et;s;c;dt]
    / YOUR CODE HERE
    t:select date, time, sym, price, size from trade where date=dt, sym in s, time within (st,et), cond in c;
    t}

In [4]:
/ list of symbols in the dow jones industria average (djia)
syms:`MMM`AXP`AAPL`BA`CAT`CVX`CSCO`KO`DIS`DWDP
syms,:`XOM`GS`HD`IBM`INTC`JNJ`JPM`MCD`MRK`MSFT
syms,:`NKE`PFE`PG`TRV`UTX`UNH`VZ`V`WMT`WBA

/ pass function to the database for execution
h (tradequery;09:30;09:35;syms;" ";2018.09.04)

date       time                 sym  price  size
------------------------------------------------
2018.09.04 0D09:30:00.075702685 AAPL 228.48 100 
2018.09.04 0D09:30:00.369093964 AAPL 228.4  700 
2018.09.04 0D09:30:00.369104482 AAPL 228.4  300 
2018.09.04 0D09:30:00.457390071 AAPL 228.4  100 
2018.09.04 0D09:30:00.575786516 AAPL 228.39 100 
..


Your results should match the following:
```
date       time                 sym  price  size
------------------------------------------------
2018.09.04 0D09:30:00.075702685 AAPL 228.48 100 
2018.09.04 0D09:30:00.369093964 AAPL 228.4  700 
2018.09.04 0D09:30:00.369104482 AAPL 228.4  300 
2018.09.04 0D09:30:00.457390071 AAPL 228.4  100 
2018.09.04 0D09:30:00.575786516 AAPL 228.39 100 
..
```

In [5]:
rnd:{x*"j"$y%x}
assert:{if[not x~y;'`$"expecting '",(-3!x),"' but found '",(-3!y),"'"]}
/ confirm all columns are included
assert[`date`time`sym`price`size] cols h (tradequery;09:30;16:00;`BAC;"O";2018.09.04)
/ confirm schema is correct
assert["dnsei"] first flip value meta h (tradequery;09:30;16:00;`BAC;"O";2018.09.01)
/ confirm query only selects specified dates
assert[1#2018.09.10]  exec distinct date from h (tradequery;09:30;16:00;`BAC;"O";2018.09.10)
/ confirm only requested sym has been returned
assert[`BAC`TSLA] exec distinct sym from h (tradequery;09:30;16:00;`BAC`TSLA;"O";2018.09.04)
/ confirm only data within the expected time range is returned
assert[1b] all within[;09:30 16:00]exec time from h (tradequery;09:30;16:00;`BAC;"O";2018.09.10)
assert[1b] all within[;09:30 12:00]exec time from h (tradequery;09:30;12:00;`BAC;"O";2018.09.10)
/ confirm only selected condition codes have been returned
assert[2] count h (tradequery;09:30;16:00;`BAC;"O";2018.09.10)
assert[7235] count h (tradequery;09:30;10:00;`BAC;" O";2018.09.10)

### Part B (1 points)

Without having actual order information, we will need to first guestimate if each trade is buyer or seller initiated using the Lee and Ready algorithm:

- If trade is closer to bid, it is seller initiated
- Else if trade is closer to ask, it is buyer initiated
- Else:
  - If last tick was an uptick, it is buyer initiated
  - Else it was seller initiated

With the trade information in hand, we can determine if each trade is an up-tick or down-tick.

Complete the `tickquery` function so that it
1. Selects trade information for a list of stocks for a single days for the given time range and trade conditions.
2. **Adds a new column `tick` that indicates if the trade was above or below the previous trade (deferring to the trades before it in the case of no change).**

In [6]:
/ (s)tart (t)ime, (e)nd (t)ime, list of (s)ym, trade (c)ondition codes, (d)a(t)e
tickquery:{[st;et;s;c;dt]
    / YOUR CODE HERE
    t:select date, time, sym, price, size from trade where date=dt, sym in s, time within (st,et), cond in c;
    t:update tick: fills -1 0N 1@1+signum price-prev[first price;price] by sym from t;
    t}

In [7]:
/ pass function to the database for execution
h (tickquery;09:30;09:35;syms;" ";2018.09.04)

date       time                 sym  price  size tick
-----------------------------------------------------
2018.09.04 0D09:30:00.075702685 AAPL 228.48 100      
2018.09.04 0D09:30:00.369093964 AAPL 228.4  700  -1  
2018.09.04 0D09:30:00.369104482 AAPL 228.4  300  -1  
2018.09.04 0D09:30:00.457390071 AAPL 228.4  100  -1  
2018.09.04 0D09:30:00.575786516 AAPL 228.39 100  -1  
..


Your results should match the following:
```
date       time                 sym  price  size tick
-----------------------------------------------------
2018.09.04 0D09:30:00.075702685 AAPL 228.48 100      
2018.09.04 0D09:30:00.369093964 AAPL 228.4  700  -1  
2018.09.04 0D09:30:00.369104482 AAPL 228.4  300  -1  
2018.09.04 0D09:30:00.457390071 AAPL 228.4  100  -1  
2018.09.04 0D09:30:00.575786516 AAPL 228.39 100  -1  
..
```

In [8]:
/ confirm all columns are included
assert[`date`time`sym`price`size`tick] cols h (tickquery;09:30;16:00;syms;"O";2018.09.04)
/ confirm schema is correct
assert["dnseij"] first flip value meta h (tickquery;09:30;09:32;syms;"O";2018.09.01)
/ confirm queried data is accurate
assert[33] count h (tickquery;09:30;09:32;syms;"O";2018.09.04)
assert[`AAPL`MSFT!39 12] `AAPL`MSFT#exec sum tick by sym from h (tickquery;09:30;09:32;syms;" ";2018.09.04)

### Part C (1 points)

To continue with our attempt to determine if each trade is buyer or seller initiated by using the Lee and Ready algorithm, we need to mark if each trade was above or below the mid price. This problem requires you to (efficiently) join the previously created `trade` query with the `nbbo` table.

Complete the `taqquery` function so that it:

1. Selects trade information for a list of stocks for a single days for the given time range and trade conditions.
2. Adds a new column `tick` that indicates if the trade was above or below the previous trade (deferring to the trades before it in the case of no change).
3. **Queries the time,sym,bsize,bid,ask, and asize columns from the `nbbo` table for the specified date.**
4. **Uses the `aj` operator to join the results of this query to the results of the trade query - making sure that the first argument to the `aj` operator includes all the necessary columns (and in the correct order).**

NOTE: In order for this query to finish in a reasonable time, Kdb+ must be able to quickly find the right quote for each trade.  Make sure your query of the nbbo table maintains a ```p`` attribute on the sym column, or apply a new one if it doesn't already have one.

In [9]:
/ (s)tart (t)ime, (e)nd (t)ime, list of (s)ym, trade (c)ondition codes, (d)a(t)e
taqquery:{[st;et;s;c;dt]
    / YOUR CODE HERE
    t:select date,time,`p#sym,price,size from trade where date=dt, sym in s, time within (st,et), cond in c;
    t:update tick: fills -1 0N 1i@1+signum price-prev[first price;price] by sym from t;
    n:select time,`p#sym,bsize,bid,ask,asize from nbbo where date=dt, sym in s;
    t:aj[`sym`time;t] n;
    t}

In [10]:
/ pass function to the database for execution
h (taqquery;09:30;09:35;syms;" ";2018.09.04)

date       time                 sym  price  size tick bsize bid    ask    asize
-------------------------------------------------------------------------------
2018.09.04 0D09:30:00.075702685 AAPL 228.48 100       1     228.4  228.49 1    
2018.09.04 0D09:30:00.369093964 AAPL 228.4  700  -1   2     228.37 228.4  10   
2018.09.04 0D09:30:00.369104482 AAPL 228.4  300  -1   2     228.37 228.4  10   
2018.09.04 0D09:30:00.457390071 AAPL 228.4  100  -1   1     228.4  228.46 3    
2018.09.04 0D09:30:00.575786516 AAPL 228.39 100  -1   1     228.31 228.42 2    
..


Your results should match the following:
```
date       time                 sym  price  size tick bsize bid    ask    asize
-------------------------------------------------------------------------------
2018.09.04 0D09:30:00.075702685 AAPL 228.48 100       1     228.4  228.49 1    
2018.09.04 0D09:30:00.369093964 AAPL 228.4  700  -1   2     228.37 228.4  10   
2018.09.04 0D09:30:00.369104482 AAPL 228.4  300  -1   2     228.37 228.4  10   
2018.09.04 0D09:30:00.457390071 AAPL 228.4  100  -1   1     228.4  228.46 3    
2018.09.04 0D09:30:00.575786516 AAPL 228.39 100  -1   1     228.31 228.42 2    
..
```

In [11]:
/ confirm all columns are included
assert[`date`time`sym`price`size`tick`bsize`bid`ask`asize] cols h (taqquery;09:30;16:00;syms;"O";2018.09.04)
/ confirm schema is correct
assert["dnseiiieei"] first flip value meta h (taqquery;09:30;09:32;syms;"O";2018.09.01)
/ confirm queried data is accurate
assert[33] count h (taqquery;09:30;09:32;syms;"O";2018.09.04)
assert[`sym`bid`ask!(`XOM;79.84e;79.87e)] `sym`bid`ask#last h (taqquery;09:30;09:32;syms;" ";2018.09.04)

### Part D (1 points)

With the trade and quote data now joined, it is possible to compute the prevailing mid price at each trade and finally determine the assumed `side` of each trade.  While we will never know the real intention of each trade (because it will always include both a buyer and sell, we can create a good estimate of the more aggressive side of the trade buy using the Lee and Ready Algorithm.

Complete the `landrquery` function so that it:

1. Selects trade information for a list of stocks for a single days for the given time range and trade conditions.
2. Adds a new column `tick` that indicates if the trade was above or below the previous trade (deferring to the trades before it in the case of no change).
3. Queries the time,sym,bsize,bid,ask, and asize columns from the `nbbo` table for the specified date.
4. Uses the `aj` operator to join the results of this query to the results of the trade query - making sure that the first argument to the `aj` operator includes all the necessary columns (and in the correct order).
5. **Adds a `mid` price column to the table (casting it to type "e" to make it comparable with the `price`column)**
6. **Adds a `side` column that is calculated by using the Lee & Ready algorithm**

Again, the Lee and Ready algorithm is defined here.  Using the `tick` column from step 2 and the mid price from step 5, you should be able to use the vector conditional operator `?[;;]` (in fact, you will use it two times) to determine the side of the trade.

- If trade is closer to bid, it is seller initiated
- Else if trade is closer to ask, it is buyer initiated
- Else:
  - If last tick was an uptick, it is buyer initiated
  - Else it was seller initiated


In [12]:
/ (s)tart (t)ime, (e)nd (t)ime, list of (s)ym, trade (c)ondition codes, (d)a(t)e
landrquery:{[st;et;s;c;dt]
    / YOUR CODE HERE
    t:select date,time,`p#sym,price,size from trade where date=dt, sym in s, time within (st,et), cond in c;
    t:update tick: fills -1 0N 1i@1+signum price-prev[first price;price] by sym from t;
    n:select time,`p#sym,bsize,bid,ask,asize from nbbo where date=dt, sym in s;
    t:aj[`sym`time;t] n;
    t:update mid: "e"$(bid+ask)%2 from t;
    t:update side: ?[price>mid;1;?[price<mid;-1;tick]] from t;
    t}

In [13]:
/ pass function to the database for execution
h (landrquery;09:30;09:35;syms;" ";2018.09.04)

date       time                 sym  price  size tick bsize bid    ask    asize mid     side
--------------------------------------------------------------------------------------------
2018.09.04 0D09:30:00.075702685 AAPL 228.48 100       1     228.4  228.49 1     228.445 1   
2018.09.04 0D09:30:00.369093964 AAPL 228.4  700  -1   2     228.37 228.4  10    228.385 1   
2018.09.04 0D09:30:00.369104482 AAPL 228.4  300  -1   2     228.37 228.4  10    228.385 1   
2018.09.04 0D09:30:00.457390071 AAPL 228.4  100  -1   1     228.4  228.46 3     228.43  -1  
2018.09.04 0D09:30:00.575786516 AAPL 228.39 100  -1   1     228.31 228.42 2     228.365 1   
..


Your results should match the following:
```
date       time                 sym  price  size tick bsize bid    ask    asize mid     side
--------------------------------------------------------------------------------------------
2018.09.04 0D09:30:00.075702685 AAPL 228.48 100       1     228.4  228.49 1     228.445 1   
2018.09.04 0D09:30:00.369093964 AAPL 228.4  700  -1   2     228.37 228.4  10    228.385 1   
2018.09.04 0D09:30:00.369104482 AAPL 228.4  300  -1   2     228.37 228.4  10    228.385 1   
2018.09.04 0D09:30:00.457390071 AAPL 228.4  100  -1   1     228.4  228.46 3     228.43  -1  
2018.09.04 0D09:30:00.575786516 AAPL 228.39 100  -1   1     228.31 228.42 2     228.365 1   
..
```

In [14]:
/ confirm all columns are included
assert[`date`time`sym`price`size`tick`bsize`bid`ask`asize`mid`side] cols h (landrquery;09:30;16:00;syms;"O";2018.09.04)
/ confirm schema is correct
assert["dnseiiieeiej"] first flip value meta h (landrquery;09:30;09:32;syms;"O";2018.09.01)
/ confirm queried data is accurate
assert[33] count h (landrquery;09:30;09:32;syms;"O";2018.09.04)
assert[([]sym:2#`XOM;side:-1 1)] `sym`side#-2#h (landrquery;09:30;09:32;syms;" ";2018.09.04)

### Part E (1 points)

We now have the assumed `side` of each trade and can begin computing minute summaries 'flow' so the size of the data returned is not very large.

Complete the `landrquerysumary` function so that it:

1. Selects trade information for a list of stocks for a single days for the given time range and trade conditions.
2. Adds a new column `tick` that indicates if the trade was above or below the previous trade (deferring to the trades before it in the case of no change).
3. Queries the time,sym,bsize,bid,ask, and asize columns from the `nbbo` table for the specified date.
4. Uses the `aj` operator to join the results of this query to the results of the trade query - making sure that the first argument to the `aj` operator includes all the necessary columns (and in the correct order).
5. Adds a `mid` price column to the table (casting it to type "e" to make it comparable with the `price`column)
6. Adds a `side` column that is calculated by using the Lee & Ready algorithm
7. **Computes the net quantity traded (buy size - sell size), and the change in mid price for each sym of each window (in minutes).  The returned table should have five columns: ```date`minute`sym`size`dprice``**

In [15]:
/ (s)tart (t)ime, (e)nd (t)ime, list of (s)ym, trade (c)ondition codes, (w)indow size in minutes, (d)a(t)e
landrquerysummary:{[st;et;s;c;w;dt]
    / YOUR CODE HERE
    t:select date,time,`p#sym,price,size from trade where date=dt, sym in s, time within (st,et), cond in c;
    t:update tick: fills -1 0N 1i@1+signum price-prev[first price;price] by sym from t;
    n:select time,`p#sym,bsize,bid,ask,asize from nbbo where date=dt, sym in s;
    t:aj[`sym`time;t] n;
    t:update mid: "e"$(bid+ask)%2 from t;
    t:update side: ?[price>mid;1;?[price<mid;-1;tick]] from t;
    t:select size: sum size*side, dprice:last mid-first mid by date,w xbar time.minute,sym from t;
    t}

In [16]:
/ pass function to the database for execution
h (landrquerysummary;09:30;09:35;syms;" ";5;2018.09.04)

date       minute sym | size   dprice    
----------------------| -----------------
2018.09.04 09:30  AAPL| 12037  0.01998901
2018.09.04 09:30  AXP | 13762  -0.4899979
2018.09.04 09:30  BA  | -12569 -0.190033 
2018.09.04 09:30  CAT | -9941  -0.1799927
2018.09.04 09:30  CSCO| 10718  0.1000023 
..


Your results should match the following:
```
date       minute sym | size   dprice    
----------------------| -----------------
2018.09.04 09:30  AAPL| 12037  0.01998901
2018.09.04 09:30  AXP | 13762  -0.4899979
2018.09.04 09:30  BA  | -12569 -0.190033 
2018.09.04 09:30  CAT | -9941  -0.1799927
2018.09.04 09:30  CSCO| 10718  0.1000023 
..
```

In [17]:
w:5 / define window size
/ confirm all columns are included
assert[`date`minute`sym`size`dprice] cols h (landrquerysummary;09:30;16:00;syms;"O";w;2018.09.04)
/ confirm schema is correct
assert["dusje"] first flip value meta h (landrquerysummary;09:30;09:32;syms;"O";w;2018.09.01)
/ confirm queried data is accurate
assert[180] count h (landrquerysummary;09:30;10:00;syms;" ";w; 2018.09.04)
assert[328831 -.12] rnd[.01] value h[(landrquerysummary;09:30;10:00;syms;" ";w;2018.09.04)](2018.09.04;09:35;`CSCO)

NYSE Daily TAQ Kdb+ tables are designed to perform well on daily queries. Our `landrquerysumary` function produces Lee and Ready summary statistics for a list of symbols with minutely buckets `w` on a single date. We can run the query over multiple dates and combine the results locally.  We don't want to run a very long function on the server, (it is not fair for everyone else and there is a per-client timeout) so we should loop over each date locally.

In [18]:
/ here comes the heavy lifting, we now query a full day's worth of data
/ and loop over 4 days.  Written efficiently, your query should only take 8.5 seconds
show t:raze {[dt]h (landrquerysummary;09:30;16:00;syms;" ";5;dt)} each 2018.09.04+til 4

date       minute sym | size   dprice    
----------------------| -----------------
2018.09.04 09:30  AAPL| 12037  0.01998901
2018.09.04 09:30  AXP | 13762  -0.4899979
2018.09.04 09:30  BA  | -12569 -0.190033 
2018.09.04 09:30  CAT | -9941  -0.1799927
2018.09.04 09:30  CSCO| 10718  0.1000023 
..


The results should match the following:
```
date       minute sym | size   dprice    
----------------------| -----------------
2018.09.04 09:30  AAPL| 12037  0.01998901
2018.09.04 09:30  AXP | 13762  -0.4899979
2018.09.04 09:30  BA  | -12569 -0.190033 
2018.09.04 09:30  CAT | -9941  -0.1799927
2018.09.04 09:30  CSCO| 10718  0.1000023 
..
```

### Part F (1 points)

With the traded volumes and resulting market moves, we can now regress Kyle's lambda from the formula:

$$\Delta m_t = \mu + \lambda q_t + \epsilon_t$$

where: 
- $\Delta m_t$ is the change in mid price over time $t$
- $\mu$ is the stock trend/drift
- $q_t$ is the order imbalance (total buy - sell orders)

This problem asks you to write a function that will regress two vectors `x` and `y` with the option of also fitting the y intercept.

The Kdb+ `lsq` operator is written to perform least squares regression over multiple dimensions.  we will only be using it on a single dimension `x` vs `y`.  Our `regress` function will accept vectors, convert them to matrices before finally return the result as a vector as well.

Complete the `regress` function so that it: 

1. Checks the `int` parameter and either converts `y` into a one-column matrix or a two-column matrix with the first column being a list of ones (if `int` is true)
2. Converts vector `x` into a one-column matrix.
3. Uses the `lsq` operator to perform least squares regression on the two matrices
4. Returns the first dimension of the resulting one-column matrix.



In [19]:
/ perform ordinary least squares regression on `x` and `y` optionally fitting the `y` (int)ercept
regress:{[int;x;y]
 / YOUR CODE HERE
 r:enlist[y] lsq ?[int;(count[x]#1f;x);enlist[x]];
 r[0;]}

In [20]:
regress[1b;1 2 3f;1 2 3f]

2.56395e-16 1


Your results should match the following:
```
2.56395e-16 1
```

In [21]:
/ confirm regression returns the correct results
assert[9h] type regress[0b;1 2 3f;1 2 3f]
assert[1] count regress[0b;1 2 3f;1 2 3f]
assert[9h] type regress[1b;1 2 3f;1 2 3f]
assert[2] count regress[1b;1 2 3f;1 2 3f]
assert[enlist 1f] rnd[.01] regress[0b;1 2 3f;1 2 3f]
assert[0 1f] rnd[.01] regress[1b;1 2 3f;1 2 3f]
assert[4 -1f] rnd[.01] regress[1b;1 2 3f;3 2 1f]

### Part G (1 points)

Let's return to the task of computing Kyle's lambda for each of the stocks in the DJIA for the few days of data stored in `t`.

Complete the `kyle` function so that it:

1. Regresses the price change column `dprice` against the net quantity traded column `size` for each `sym` in the table and stores the results in a `results` column. The values in the `results` column will be lists which we will need to 'pick apart'.
2. If `int` is true, adds a `mu` column that has the first value of **each** of the `results` rows.
3. Adds a `lambda` column that has the last value of **each** of the `results` rows.
4. Uses the `delete` operator to remove the `results` column.

The function will accept two parameters, `int` which specifies whether we want to fit the stock drift and `t` which will be a table with the sym, size and dprice columns.

The resulting table should have two (or three) columns. If `int` is true, the table will have sym, mu and lambda columns, otherwise it will only have sym and lambda.

NOTE: because Kdb+ overloads operators based on their type, it is important that the values in the matrices passed to the `lsq` operator are all "f"loats.  You will get a type error if this is not the case.

In [22]:
/ perform ordinary least squares regression on `x` and `y` optionally fitting the `y` (int)ercept
kyle:{[int;t]
 / YOUR CODE HERE
 t: select result: enlist[dprice] lsq ?[int;(count[size]#1f;size); enlist(size)] by sym from t;
 if[int;t:update mu: result[;0;0],lambda:result[;0;1] from t];
 if[not int;t:update lambda:result[;0;0] from t];
 t: delete result from t;
 t}

In [23]:
kyle[1b;update "f"$dprice,"f"$size from t] / make sure to pass "f"loat values

sym | mu           lambda      
----| -------------------------
AAPL| 0.02512178   6.412626e-06
AXP | 0.007169901  2.732008e-06
BA  | 0.04999768   3.609343e-05
CAT | 0.01375689   1.367268e-06
CSCO| -0.001225552 6.537366e-08
..


Your results should match the following:
```
sym | mu           lambda      
----| -------------------------
AAPL| 0.02512178   6.412626e-06
AXP | 0.007169901  2.732008e-06
BA  | 0.04999768   3.609343e-05
CAT | 0.01375689   1.367268e-06
CSCO| -0.001225552 6.537366e-08
..
```

In [24]:
/ confirm kyle returns the correct results
assert[(30;`sym`lambda)] (count;cols)@\: kyle[0b;update "f"$dprice,"f"$size from t]
assert[(30;`sym`mu`lambda)] (count;cols)@\:kyle[1b;update "f"$dprice,"f"$size from t]
assert[(1#`lambda)!enlist 0.06 0.02] flip rnd[.01] 1e4*2#value kyle[0b;update "f"$dprice,"f"$size from t]
assert[`mu`lambda!(251.22 71.7;0.06 0.03)] flip rnd[.01] 1e4*2#value kyle[1b;update "f"$dprice,"f"$size from t]

### Part H (1 points)

With the regression coefficients calculated, we can now compute the expected price movement (impact) for each size in the multi-day Lee and Ready summary table `t`.

For this problem you should:

1. Join the results of the `kyle` query (stored in the `k` variable) with the Lee and Ready summary table `t`
2. Add a new `impact` column which uses the net traded `size` in the time bucket with the computed `lambda` for that stock to 'predict' the price move

NOTE: This 'prediction' is completely *in sample* and should not be taken literally.


In [25]:
k:kyle[0b;update "f"$dprice,"f"$size from t]
/ YOUR CODE HERE
t:t lj k
t:update impact: lambda*size from t

In [26]:
t

date       minute sym | size   dprice     lambda       impact      
----------------------| -------------------------------------------
2018.09.04 09:30  AAPL| 12037  0.01998901 6.147495e-06 0.07399739  
2018.09.04 09:30  AXP | 13762  -0.4899979 2.461496e-06 0.03387511  
2018.09.04 09:30  BA  | -12569 -0.190033  3.495276e-05 -0.4393212  
2018.09.04 09:30  CAT | -9941  -0.1799927 1.292013e-06 -0.0128439  
2018.09.04 09:30  CSCO| 10718  0.1000023  7.144101e-08 0.0007657048
..


Your results should match the following:
```
date       minute sym | size   dprice     lambda       impact      
----------------------| -------------------------------------------
2018.09.04 09:30  AAPL| 12037  0.01998901 6.147495e-06 0.07399739  
2018.09.04 09:30  AXP | 13762  -0.4899979 2.461496e-06 0.03387511  
2018.09.04 09:30  BA  | -12569 -0.190033  3.495276e-05 -0.4393212  
2018.09.04 09:30  CAT | -9941  -0.1799927 1.292013e-06 -0.0128439  
2018.09.04 09:30  CSCO| 10718  0.1000023  7.144101e-08 0.0007657048
..
```

In [27]:
/ confirm impact 'prediction' was performed correctly
assert[`date`minute`sym`size`dprice`lambda`impact] cols t
assert[9360] count t
assert[12037 0.02 0 0.07] rnd[.01] value t[(2018.09.04;09:30;`AAPL)]
assert[-14557 0.25 0 -0.01] rnd[.001] value t[(2018.09.07;09:40;`WMT)]


### Part I (1 points)

In addition to being a measure of market impact, Kyle's lambda can also be interpreted as a liquidity (or inverse liquidity to be more precise) measure.

For this problem you should sort the table `k` so that the most liquid stock is at the top, and the least liquid stock is at the bottom.

A few questions to consider:
- Would fitting the stock drift change the results?
- What do you think the R^2 of these regressions would be?
- Is a linear relationship between the quantity traded and price change realistic?

In [28]:
/ YOUR CODE HERE
k:kyle[0b;update "f"$dprice,"f"$size from t]
k:select lambda by sym from k
k:`lambda xasc k

In [29]:
k

sym | lambda      
----| ------------
CSCO| 7.144101e-08
INTC| 7.882344e-08
KO  | 1.796685e-07
PFE | 1.98921e-07 
VZ  | 2.449397e-07
..


Your results should match the following:
```
sym | lambda      
----| ------------
CSCO| 7.144101e-08
INTC| 7.882344e-08
KO  | 1.796685e-07
PFE | 1.98921e-07 
VZ  | 2.449397e-07
..
```

In [30]:
/ confirm `k` was properly sorted by impact
assert[`BA] last last key k
assert[`CSCO] last first key k


### Part J (1 points)

Market impact is not - in fact - linear with the quantity traded.  The Square Root 'Law' has been shown to be a good  estimate of the market impact, given a stock's average daily volume and volatility.

$$ I(Q)\sim\sigma\left(\frac{\mid Q\mid}{V}\right)^\frac{1}{2}$$


This question requests that you determine the impact expected for trading USD \$1M of each of the DJIA stocks.

Complete the `impact` function so that it:

1. Selects the sym,size and close data for the given time window and symbols
2. Computes the average daily volume `adv`, daily volatility `vol` and last price `close` for each symbol
3. Adds a `qty` column which is derived from the provided `n`otional and last `close` price
4. Adds a column with the impact as a percent `impactpct` using the square root law (assuming proportionality 1)
5. Adds a column with the impact as a price `impactpx`

The resulting table should have the following columns: sym, adv, vol, close, qty, impactpct, impactpx.

NOTE: make sure to use the `sdev` function on daily **log** returns when computing volatility

In [31]:
/ computes the expected impact (in dollars) for each (s)ym when trading (n)otional
/ average daily volume and standard deviation of each (s)ym are computed between (s)tart (d)ate and (e)nd (d)ate
impact:{[n;sd;ed;s]
/ YOUR CODE HERE
 t:select date,time,`p#sym,price,size from trade where date within (sd;ed), sym in s;
 t:select close: last price, size: sum size by sym, date from t;
 t:select adv: avg size, vol: sdev log close%prev close, close: last close by sym from t;
 t:update qty: n%close from t;
 t:update impactpct: vol*sqrt[abs qty%adv] from t;
 t:update impactpx: impactpct*close from t;
 t}

In [32]:
show i:h (impact;1e6;2018.09.04;2018.09.30;syms)

sym | adv          vol         close  qty      impactpct    impactpx  
----| ----------------------------------------------------------------
AAPL| 4.031043e+07 0.0160347   225.93 4426.15  0.0001680216 0.03796112
AXP | 3510807      0.009031966 106.89 9355.412 0.0004662408 0.04983648
BA  | 3753490      0.01145201  372.4  2685.285 0.0003063086 0.1140693 
CAT | 4552661      0.01159729  152.5  6557.377 0.0004401381 0.06712106
CSCO| 2.284504e+07 0.00887877  48.73  20521.24 0.0002661084 0.01296746
..


Your results should match the following:
```
sym | adv          vol         close  qty      impactpct    impactpx  
----| ----------------------------------------------------------------
AAPL| 4.031043e+07 0.0160347   225.93 4426.15  0.0001680216 0.03796112
AXP | 3510807      0.009031966 106.89 9355.412 0.0004662408 0.04983648
BA  | 3753490      0.01145201  372.4  2685.285 0.0003063086 0.1140693 
CAT | 4552661      0.01159729  152.5  6557.377 0.0004401381 0.06712106
CSCO| 2.284504e+07 0.00887877  48.73  20521.24 0.0002661084 0.01296746
..
```

In [33]:
/ confirm the table columns are correct
assert[`sym`adv`vol`close`qty`impactpct`impactpx] cols i
/ confirm the number of rows is correct
assert[30] count i
/ confirm adv is computed properly
assert[6132332 7547803] rnd[1] i[([]sym:`IBM`JNJ);`adv]
/ confirm volatility is computed properly
assert[0.0074 0.008] rnd[1e-4] i[([]sym:`IBM`JNJ);`vol]
/ confirm qty is computed properly
assert[6610 7237] rnd[1] i[([]sym:`IBM`JNJ);`qty]
/ confirm impactpct is computed properly
assert[0.000242 0.000248] rnd[1e-6] i[([]sym:`IBM`JNJ);`impactpct]
/ confirm impactpx is computed properly
assert[0.0366 0.0342] rnd[1e-4] i[([]sym:`IBM`JNJ);`impactpx]